# 임베딩

In [ ]:
from openai import OpenAI

In [17]:
client = OpenAI()

text = '이것은 테스트입니다.'
model = 'text-embedding-ada-002'

response = client.embeddings.create(input = [text],
                                    model = model)

In [18]:
client = OpenAI()

text1 = '이것은 테스트입니다.'
text2 = '이것은 실전입니다.'
model = 'text-embedding-ada-002' # 해당 모델이 생성하는 임베딩의 차원은 1536, 최대 토큰수는 8191

response = client.embeddings.create(input = [text1, text2],
                                    model = model)

# response.data <- list의 형태로, input 값에 대한 임베딩 배열이 리스트 원소에 들어있다.
# response.data[0].embedding # 1536 차원 벡터로 변환

### 유사도 검색

In [39]:
import openai
import numpy as np
import faiss

In [42]:
client = OpenAI()

in_text = '오늘은 비가 오지 않아서 다행이다.'
model = 'text-embedding-ada-002'

response = client.embeddings.create(input = [in_text],
                                    model = model)

in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype('float32')

In [45]:
target_texts = [
                '좋아하는 음식은 무엇인가요?',
                '어디에 살고 계신가요?',
                '아침 전철은 혼잡하네요',
                '오늘은 날씨가 좋네요!',
                '요즘 경기가 좋지 않네요.'
]

response = client.embeddings.create(input = target_texts,
                                    model = model)

target_embeds = [record.embedding for record in response.data]
target_embeds = np.array(target_embeds).astype('float32')

In [46]:
index = faiss.IndexFlatL2(1536)

In [47]:
index.add(target_embeds)

In [49]:
D, I = index.search(in_embeds, 1)

print(D) # distance
print(I) # index
print(target_texts[I[0][0]])

[[0.33448073]]
[[3]]
오늘은 날씨가 좋네요!


# 파인튜닝

In [54]:
import pandas as pd

In [57]:
df = pd.read_csv('./tsukuyomi.csv',
                 usecols = [1, 2],
                 names = ['prompt', 'completion'],
                 skiprows = 2)

In [59]:
# 데이터 프레임을 json형태로 변형(학습 데이터에서 요구하는 포맷)
df.to_json('./tsukuyomi.jsonl',
           orient = 'records',
           lines = True,
           force_ascii = False)

'''
    {'prompt': '프롬프트 예시1', 'completion': '컴플리션 예시1'}
    {'prompt': '프롬프트 예시2', 'completion': '컴플리션 예시2'}
'''

In [125]:
def format_chat(row):
    result = json.dumps(
                        {'messages': [
                                        {'role': 'system', 'content': '츠쿠요미는 여성 캐릭터 입니다.'},
                                        {'role': 'user', 'content': row['prompt']},
                                        {'role': 'assistant', 'content': row['completion']}
                        ]}
    )
    
    return result
    
df['conversation'] = df.apply(format_chat, axis = 1)

In [126]:
with open('./tsukuyomi_prepared.jsonl', 'w') as jsonl_file:
    for row in df['conversation']:
        jsonl_file.write(row + '\n')

In [ ]:
# with open('./tsukuyomi_prepared.jsonl', 'r') as f:
#     test_file = [json.loads(line) for line in f]

In [119]:
client = OpenAI()

In [127]:
file = client.files.create(file = open('./tsukuyomi_prepared.jsonl', 'rb'),
                           purpose = 'fine-tune')

In [128]:
print(file)

FileObject(id='file-3UrWgFhnkmz67e6PpZNjhUMj', bytes=184146, created_at=1701829522, filename='tsukuyomi_prepared.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [129]:
client.files.retrieve(file.id)

FileObject(id='file-3UrWgFhnkmz67e6PpZNjhUMj', bytes=184146, created_at=1701829522, filename='tsukuyomi_prepared.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [130]:
client.fine_tuning.jobs.create(training_file = file.id,
                               model = 'gpt-3.5-turbo')

BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}